In [1]:
from bs4 import BeautifulSoup
import lxml
import requests
import re
import os
from tqdm import tqdm
from datetime import datetime

In [23]:
cnt = requests.get('https://myanimelist.net/anime/42938/Fruits_Basket__The_Final')

soup = BeautifulSoup(cnt.content, features="lxml")
print(soup.prettify())

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html lang="en" xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#">
 <head>
  <link crossorigin="anonymous" href="//fonts.gstatic.com/" rel="preconnect"/>
  <link crossorigin="anonymous" href="//fonts.googleapis.com/" rel="preconnect"/>
  <link crossorigin="anonymous" href="//tags-cdn.deployads.com/" rel="preconnect"/>
  <link crossorigin="anonymous" href="//www.googletagservices.com/" rel="preconnect"/>
  <link crossorigin="anonymous" href="//www.googletagmanager.com/" rel="preconnect"/>
  <link crossorigin="anonymous" href="//apis.google.com/" rel="preconnect"/>
  <link crossorigin="anonymous" href="//pixel-sync.sitescout.com/" rel="preconnect"/>
  <link crossorigin="anonymous" href="//pixel.tapad.com/" rel="preconnect"/>
  <link crossorigin="anonymous" href="//c.deployads.com/" rel="preconnect"/>
  <link crossorigin="anonymous" href="//tpc.

In [31]:
cnt = requests.get('https://myanimelist.net/anime/42938/Fruits_Basket__The_Final')

soup = BeautifulSoup(cnt.content, features="lxml")
info = soup.find_all('div',{"class":"spaceit_pad"})

lis = [text for text in soup.stripped_strings]
animeTitle = soup.strong.string
for index,word in enumerate(lis):
  if word == 'Type:':
    animeType = str(lis[index+1])
    lis.remove('Type:')
  if word == 'Episodes:':
    animeNumEpisode = str(lis[index+1])
    lis.remove('Episodes:')
  if word == 'Aired:':
    prepro = str(lis[index+1])
    releaseDate,endDate = prepro.split('to')
    releaseDate = datetime.strptime(releaseDate,"%b %d, %Y ")
    endDate = datetime.strptime(endDate," %b %d, %Y")
    lis.remove('Aired:')
  if word == 'Members:':
    animeNumMembers = str(lis[index+1])
    animeNumMembers = animeNumMembers.replace(',','')
    animeNumMembers = int(animeNumMembers)
    lis.remove('Members:')
  if word == 'Score:':
    animeScore = float(lis[index+1])
    lis.remove('Score:')
  if word == '(scored by':
    animeUsers = int(lis[index+1])
    lis.remove('(scored by')
  if word == 'Ranked:':
    animeRank = str(lis[index+1])
    animeRank = animeRank[1:]
    animeRank = float(animeRank)
    lis.remove('Ranked:')
  if word == 'Popularity:':
    animePopularity = int(lis[index+1])
    lis.remove('Popularity:')


print(animeTitle,animeType,animeNumEpisode,releaseDate,endDate,animeNumMembers,animeScore,animeUsers,animeRank)

print(lis)
  
  




ValueError: invalid literal for int() with base 10: '#654'

In [6]:
cnt = requests.get('https://myanimelist.net/anime/5114/Fullmetal_Alchemist__Brotherhood')
soup = BeautifulSoup(cnt.content, features="lxml")
animeStaffs = []
Staffs= soup.findAll("a", { "class" : "fw-n"})
len_info = len(Staffs)
#print(Staffs)
for i in range(len_info):
    tag = (Staffs[i].find('img')).get("alt")
    animeStaffs.append(tag)
#print(animeStaffs)
for i in animeStaffs:        
     print(i)

Elric, Edward
Elric, Alphonse
Mustang, Roy
Hughes, Maes
Greed
Hawkeye, Riza
Yao, Ling
Armstrong, Alex Louis
Rockbell, Winry
Armstrong, Olivier Mira
Cook, Justin
Yonai, Noritomo
Irie, Yasuhiro
Mima, Masafumi


In [32]:
def info_extractor(page):
    cnt = requests.get(page)
    soup = BeautifulSoup(cnt.content,features='lxml')
  
  # extract all strings in the html
    lis = [text for text in soup.stripped_strings] 
  # title is incapsulated in the strong attr
    animeTitle = soup.strong.string
  # we initialize empty strings for missing values
    animeType = ''
    animeNumEpisode = ''
    releaseDate = ''
    endDate = ''
    animeNumMembers = ''
    animeScore = ''
    animeUsers = ''
    animeRank = ''
    animePopularity = ''
    
    
    
    
    
  # look for words inside lis, for example the string after "Episodes:" contains the number of episodes
  # we remove it afterwards to avoid overwriting and other potential issues
  # we also did some pre processing of data in some cases
    for index,word in enumerate(lis):
        if word == 'Type:':
            animeType = str(lis[index+1])
            try:
                lis.remove('Type:')
            except ValueError:
                 pass
    
        if word == 'Episodes:':
            animeNumEpisode = int(lis[index+1])
            lis.remove('Episodes:')

    
    # in here we convert dates to datetime
        if word == 'Aired:':
            prepro = str(lis[index+1])
            releaseDate,endDate = prepro.split('to')
            releaseDate = datetime.strptime(releaseDate,"%b %d, %Y ")
            endDate = datetime.strptime(endDate," %b %d, %Y")
            try:
                lis.remove('Aired:')
            except ValueError:
                 pass
    
    # members number is written using commas, so we replace them
        if word == 'Members:':
            animeNumMembers = str(lis[index+1])
            animeNumMembers = animeNumMembers.replace(',','')
            animeNumMembers = int(animeNumMembers)
            try:
                lis.remove('Members:')
            except ValueError:
                 pass
    
        if word == 'Score:':
            animeScore = float(lis[index+1])
            try:
                lis.remove('Score:')
            except ValueError:
                 pass
    
        if word == '(scored by':
            animeUsers = int(lis[index+1])
            try:
                lis.remove('(scored by')
            except ValueError:
                 pass
    
    # rank is always preceded by an '#' symbol, we remove it
        if word == 'Ranked:':
            animeRank = str(lis[index+1])
            animeRank = float(animeRank[1:])
            try:
                lis.remove('Ranked:')
            except ValueError:
                 pass
    
        if word == 'Popularity:':
            animePopularity = str(lis[index+1])
            animePopularity = float(animePopularity[1:])
            try:
                lis.remove('Popularity:')
            except ValueError:
                 pass
    
    # description is found after the description attr, we concatenate the strings of the synopsis
    animeDescription=''
    for i in soup.find('p',itemprop='description').stripped_strings:
        animeDescription+=i
    
    # we isolate the related animes with the attrs, look for hrefs and avoid putting multiple copies
    animeRelated=[]
    related=soup.find(class_="anime_detail_related_anime")
    iter = related.find_all(class_='borderClass')
    for i in iter:
        if i.find('href'==True) and i.string not in animeRelated:
            animeRelated.append(i.string)
    
    # again, we isolate characters, voices and staff and then append them inside lists
    animeCharacters = []
    chars = soup.findAll("h3", {"class" :"h3_characters_voice_actors"})
    len_chars = len(chars)
    for i in range(len_chars):
        animeCharacters.append(chars[i].string) 

    animeVoices = []
    voices = soup.findAll("td", {"class" :"va-t ar pl4 pr4"})
    len_voices = len(voices)
    for i in range(len_voices):
        animeVoices.append(voices[i].find('a').string)
    
    return animeTitle, animeType, animeNumEpisode, releaseDate, endDate, animeNumMembers, animeScore, animeUsers, animeRank, animePopularity, animeDescription, animeRelated, animeCharacters, animeVoices
    
    

    


  


In [33]:
info_extractor('https://myanimelist.net/anime/42938/Fruits_Basket__The_Final')

ValueError: invalid literal for int() with base 10: '/'